In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

In [35]:
df = pd.read_csv("https://raw.githubusercontent.com/ajiteshshree/Spaceship-Titanic/main/spaceship-titanic/train.csv")
df_test = pd.read_csv("https://raw.githubusercontent.com/ajiteshshree/Spaceship-Titanic/main/spaceship-titanic/test.csv")

In [36]:
pid = df['PassengerId'].tolist()
pid2 = df_test['PassengerId'].tolist()

In [37]:
rows = df.shape[0]
j=0
while j < rows:
    pid[j] = pid[j][0:4]
    j +=1
rows2 = df_test.shape[0]
k=0
while k < rows2:
    pid2[k] = pid2[k][0:4]
    k+=1

In [38]:
original_pid = df['PassengerId'].tolist()
original_pid2 = df_test['PassengerId'].tolist()

In [39]:
df['PassengerId'] = df['PassengerId'].replace(original_pid,pid)
df_test['PassengerId'] = df_test['PassengerId'].replace(original_pid2,pid2)

In [40]:
df.drop('Name',axis=1,inplace=True)
df_test.drop('Name',axis=1,inplace=True)

In [41]:
print("df.shape : ", df.shape)
print("df_test.shape : ", df_test.shape)

df.shape :  (8693, 13)
df_test.shape :  (4277, 12)


In [42]:
#removing luxiries from analysis
df.drop(['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck'],inplace=True,axis=1)
df_test.drop(['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck'],inplace=True,axis=1)

In [43]:
#filling NaN values
def preprocess(df):
    li = ['Age']
    for col in li:
        df[col].fillna(df[col].mean(),inplace=True)
    df["HomePlanet"].fillna(df["HomePlanet"].mode()[0],inplace=True)
    df["CryoSleep"].fillna(df["CryoSleep"].mode()[0],inplace=True)
    df["Destination"].fillna(df["Destination"].mode()[0],inplace=True)
    df["VIP"].fillna(df["VIP"].mode()[0],inplace=True)
    df["Cabin"].fillna("",inplace=True)

In [44]:
preprocess(df)
preprocess(df_test)

In [45]:
print("Number of nulls in cabin : ",df['Cabin'].isnull().sum())
df['Cabin'].mode()

Number of nulls in cabin :  0


0    
Name: Cabin, dtype: object

In [46]:
# df = df[df['Cabin'].notna()]

In [47]:
y = np.array(df["Transported"])
df.drop(['Transported'], axis=1, inplace=True)

In [48]:
df.isnull().sum()

PassengerId    0
HomePlanet     0
CryoSleep      0
Cabin          0
Destination    0
Age            0
VIP            0
dtype: int64

In [49]:
df_test.isnull().sum()

PassengerId    0
HomePlanet     0
CryoSleep      0
Cabin          0
Destination    0
Age            0
VIP            0
dtype: int64

In [50]:
def slice_cabin(df):
    b=0
    cabin_deck =[]
    cabin_side = []
    while b < df.shape[0]:
        if df.iloc[b,3] != "":
            cabin_deck.append(df.iloc[b,3][0])
            cabin_side.append(df.iloc[b,3][-1])
        else:
            cabin_deck.append("")
            cabin_side.append("")
        b += 1
    df['cabin_deck'] = cabin_deck
    df['cabin_side'] = cabin_side
    df.drop('Cabin', axis=1, inplace=True)

In [51]:
slice_cabin(df)
slice_cabin(df_test)

In [52]:
df_test.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,cabin_deck,cabin_side
0,0013,Earth,True,TRAPPIST-1e,27.0,False,G,S
1,0018,Earth,False,TRAPPIST-1e,19.0,False,F,S
2,0019,Europa,True,55 Cancri e,31.0,False,C,S
3,0021,Europa,False,TRAPPIST-1e,38.0,False,C,S
4,0023,Earth,False,TRAPPIST-1e,20.0,False,F,S


In [53]:
cols = df.columns
cols2 = df_test.columns

In [54]:
# for x in cols:
#         print(x,end = ' : ')
#         print(df[x].unique())
# print()
# for t in cols2:
#     print(t,end = ' : ')
#     print(df_test[t].unique())       

In [55]:
# df = df[['cabin_deck','cabin_side','VIP','CryoSleep','HomePlanet','Destination','Age','PassengerId']]

In [56]:
# df_test = df_test[['cabin_deck','cabin_side','VIP','CryoSleep','HomePlanet','Destination','Age','PassengerId']]

In [57]:
cols = df.columns
cols2 = df_test.columns

In [58]:
le = LabelEncoder()

In [59]:
id = df['PassengerId'].tolist()
int_id = []
for x in id:
    int_id.append(int(x))

id2 = df_test['PassengerId'].tolist()
int_id2 = []
for x in id2:
    int_id2.append(int(x))

In [60]:
df['PassengerId'] = df['PassengerId'].replace(id,int_id)
df_test['PassengerId'] = df_test['PassengerId'].replace(id2,int_id2)

In [61]:
non_int_cols = [k for k, v in dict(df.dtypes).items() if v == 'O' or v == 'bool']

In [62]:
non_int_cols

['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'cabin_deck', 'cabin_side']

In [63]:
# label encode
for x in non_int_cols:
    df[x] = le.fit_transform(df[x])
        
for x in non_int_cols:
    df_test[x] = le.fit_transform(df_test[x])

In [64]:
df = pd.get_dummies(df, columns=non_int_cols)
df_test = pd.get_dummies(df_test, columns=non_int_cols)

In [67]:
df.shape

(8693, 24)

In [68]:
logisticRegr = LogisticRegression(max_iter=7000)

In [69]:
df_test.shape

(4277, 24)

In [70]:
np.unique(y,return_counts=True)

(array([False,  True]), array([4315, 4378], dtype=int64))

In [71]:
logisticRegr.fit(df,y)

LogisticRegression(max_iter=7000)

In [72]:
res = logisticRegr.predict(df_test).tolist()

In [73]:
logisticRegr.score(df, y)

0.7225353732888531

In [74]:
predictions = pd.DataFrame({'PassengerId' : original_pid2,'Transported':res})

In [75]:
predictions.set_index('PassengerId')

,Transported
PassengerId,
0013_01,True
0018_01,False
0019_01,True
0021_01,True
0023_01,False
...,...
9266_02,True
9269_01,False
9271_01,True


In [76]:
predictions.to_csv('predictions.csv',index = False)